# Préparation des données

- trouver/supprimer les données dupliquées : df.duplicated() / df.drop_duplicates()
- renommer des colonnes : df.rename(columns={...))
- trouver les NaN : df.isnull() / df.notnull() / df.dropna()
- travail sur les chaines : series.str.extract(), series.str.contains(), series.get_dummies()
- mapping : series.map()
- changer le type d'une série (cast) : df.astype(type) / pd.to_numeric() / pd.to_datetime()
- remplacer n'importe quelle valeur : df.replace({...})
- remplacer les NaN : df.fillna(), series.combine_first()

In [ ]:
# imports
import numpy as np
import pandas as pd

In [ ]:
%pycat people.csv

#### Chargement et analyse des données

In [ ]:
# load data
df = pd.read_csv('people.csv')
df

In [ ]:
# info
df.info()

#### Duplications

- duplicated() : True ou False selon si une ligne est dupliquée
- drop_duplicates() : suppression des lignes dupliquées

In [ ]:
# lignes dupliquées
df.duplicated().value_counts()

In [ ]:
# toutes les lignes dupliquées
df.loc[df.duplicated(keep=False)].sort_values('id')

In [ ]:
# suppression des lignes dupliquées
df = df.drop_duplicates()
len(df)

#### Renommage de la colonne 'email address'

In [ ]:
# renommer les colonnes
df = df.rename(columns={'email address': 'email'})
df.head()

#### Analyse des données manquantes

numpy.nan est utilisé dans pandas pour représenter des valeurs manquantes.

In [ ]:
# Not A Number
np.nan

In [ ]:
# type
type(np.nan)

In [ ]:
# élément super absorbant
np.nan + 1

In [ ]:
# élément super absorbant
np.sqrt(np.nan)

In [ ]:
# élément super absorbant
np.nan == np.nan

In [ ]:
# au passage infinis numpy
np.NINF, np.inf

#### Tests sur les données manquantes

- isnull() ou isna()
- notnull() ou notna()

In [ ]:
# chercher les first_name Nan
df.loc[df['first_name'].isnull()]

In [ ]:
# chercher tous les lignes avec au moins un NaN
df.loc[df.isnull().any(axis=1)]

In [ ]:
# suppression de toutes les lignes avec un NaN
df.dropna()

In [ ]:
# suppression de toutes les lignes avec un NaN
df.dropna().isnull().any().any()

In [ ]:
# supprimer uniquement les lignes dont le first_name NaN 
df = df.dropna(subset=['first_name'])
len(df)

#### Ajout d'une colonne 'full_name'

In [ ]:
# 'full_name'  = 'first_name last_name'
df['full_name'] = df['first_name'] + ' ' + df['last_name']

#### Analyse de la colonne 'address'

In [ ]:
# analyse de address
df['address'].value_counts()

#### Ajout des colonnes 'city' et 'country'

In [ ]:
# calcul de city et country à partir de address
df[['city', 'country']] = df['address'].str.extract('(.*), (.*)')
df.head()

#### Analyse des données

In [ ]:
# nunique : modalités par colonne
df.nunique()

#### Mapping du genre

In [ ]:
# analyse du gender
df['gender'].unique()

In [ ]:
# analyse du gender
df['gender'].value_counts()

In [ ]:
# traitement du gender
mapping = {'Female': 'F', 'Male': 'M'}
df['gender'] = df['gender'].map(mapping)
df.head()

In [ ]:
# traitement du gender, map() avec une Series
mapping = pd.Series({'Female': 'F', 'Male': 'M'})
mapping

In [ ]:
# traitement du gender avec une Series
df['gender'] = df['gender'].map(mapping)

#### Analyse du genre

In [ ]:
# analyse gender NaN
len(df.loc[df['gender'].isnull()])

In [ ]:
# analyse prénom avec gender NaN
df.loc[df['gender'].isnull(), 'first_name'].nunique()

In [ ]:
# analyse gender
df.loc[df['gender'].isnull(), 'first_name'].unique()

Compléter le genre :

1. Autocomplétion avec le fichier people.csv (mais très peu de cas)
2. Gender API : https://gender-api.com/fr (simple mais API payante si gros volumes + de 500/mois)
3. US SSA baby names : https://www.ssa.gov/oact/babynames/limits.html ("gratuit", stats à produire, éventuellement affiner par année de naissance)

In [ ]:
# 1) Fichier people.csv
for first_name in df.loc[df['gender'].isnull(), 'first_name'].unique():
    print(first_name, df.loc[df['first_name']==first_name, 'gender'].unique())

#### Analyse de l'âge

pandas.Series.astype() : types

pandas.to_numeric() : data avec gestion des erreurs

In [ ]:
# analyse de l'âge
df.loc[df['age'].str.contains('[^0-9]'), 'age'].value_counts()

In [ ]:
# traitement de l'âge
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df

#### Traitement des dates


pandas.to_datetime() : data, gestion des formats et des erreurs

pandas.Series.combine_first() : équivalent à fillna()

In [ ]:
# conversion des dates
df['registration'] = pd.to_datetime(df.registration)
df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
# si last_seen est NaN, prendre registration
df['last_seen'] = df['last_seen'].fillna(df['registration'])
# idem
df['last_seen'] = df['last_seen'].combine_first(df['registration'])

#### Analyse de currency

In [ ]:
# échantillon
np.random.seed(0)
df.sample(10)

#### Traitement de 'currency'

Produire une nouvelle colonne numérique 'money_eur'.

Pour la conversion USD/EUR, on utilise l'API https://api.exchangeratesapi.io/latest

In [ ]:
# API
import json

import requests

response = requests.get('https://api.exchangeratesapi.io/latest')
rates = json.loads(response.content)
rates

In [ ]:
# extraction de la currency
df['currency'] = df['money'].str[0].map({'€': 'EUR', '$': 'USD'})
df['money_eur'] = df['money'].str[1:].str.replace(',', '.')  # extraction des derniers chars + , => .
df['money_eur'] = pd.to_numeric(df['money_eur'])  # conversion en nombre

# conversion des monnaies en euros
rates['rates']['EUR'] = 1.0
df['money_eur'] = df['money_eur'] * df['currency'].map(rates['rates'])
np.random.seed(0)
df.sample(10)

#### Analyse des emails

On va utiliser des regex pour nettoyer les emails mais mieux vaut utiliser une librairie spécialisée. Par exemple, https://github.com/syrusakbary/validate_email

In [ ]:
# email NaN
df['email'].isnull().sum()

In [ ]:
# suppression des emails absents
df = df.dropna(subset=['email']).copy()

In [ ]:
# emails avec chars non admis
df.loc[df['email'].str.contains('[^A-Za-z0-9_\-%+.@]'), 'email'].values

In [ ]:
# suppression des blancs
df['email'] = df['email'].str.strip()
df.loc[df['email'].str.contains('[^A-Za-z0-9_\-%+.@]'), 'email']

In [ ]:
# regex pour vérifier les domaines
df.loc[~df['email'].str.contains('.+@[A-Za-z0-9_\-.]+\.[A-Za-z]{2,}')]

In [ ]:
# emails avec noms de domaine invalides
df = df.loc[df['email'].str.contains('.+@[A-Za-z0-9_\-.]+\.[A-Za-z]{2,}')]

In [ ]:
# emails avec aliases (char +)
df.loc[df['email'].str.contains('\+'), 'email'].values

In [ ]:
# extraction des aliases (char +)
tab_email = df['email'].str.extract('([^+]+)(\+.*)?(@.+)').sort_values(0)
tab_email

In [ ]:
# suprresion des aliases (char +)
df['email'] = tab_email[0] + tab_email[2]
df.sort_values('email')

In [ ]:
# suppression des emails en double, on conserve la première ligne
df = df.drop_duplicates(subset=['email'])
df.sort_values('email')

#### Analyse de la colonne 'preference'

In [ ]:
# analyse de preference
df['preference'].nunique()

In [ ]:
# analyse de preference
df['preference'].value_counts()

In [ ]:
# modalités de preference
s = set()
df['preference'].apply(lambda x: s.update(x.split('/')))
s

In [ ]:
# ajour d'un booléen par preference
for x in sorted(s):
    df[x] = df['preference'].str.contains(x)
    
df

In [ ]:
# autre façon avec get_dummies
df['preference'].str.get_dummies(sep='/')

In [ ]:
# assignation des préférences
tab_preference = df['preference'].str.get_dummies(sep='/')
df[tab_preference.columns] = tab_preference.astype(bool)
df

#### clean

In [ ]:
import json

import requests
import pandas as pd

def clean_people(df):
    
    # suppression des lignes dupliquées
    df = df.drop_duplicates()
    
    # renommer les colonnes
    df = df.rename(columns={'email address': 'email'})
    
    # supprimer uniquement les lignes dont le first_name vaut NaN 
    df = df.dropna(subset=['first_name'])
    
    # ajout d'une colonne 'full_name'
    df['full_name'] = df['first_name'] + ' ' + df['last_name']

    # calcul de city et country à partir de address
    df[['city', 'country']] = df['address'].str.extract('(.*), (.*)')

    # traitement du gender
    mapping = {'Female': 'F', 'Male': 'M'}
    df['gender'] = df['gender'].map(mapping)
 
    # traitement de l'âge
    df['age'] = pd.to_numeric(df['age'], errors='coerce')

    # conversion des dates
    df['registration'] = pd.to_datetime(df.registration)
    df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
    # si last_seen est NaN, prendre registration
    df['last_seen'] = df['last_seen'].fillna(df['registration'])

    # récupération des taux de change
    response = requests.get('https://api.exchangeratesapi.io/latest')
    rates = json.loads(response.content)

    # extraction de la currency
    df['currency'] = df['money'].str[0].map({'€': 'EUR', '$': 'USD'})
    df['money_eur'] = df['money'].str[1:].str.replace(',', '.')  # extraction des derniers chars + , => .
    df['money_eur'] = pd.to_numeric(df['money_eur'])  # conversion en nombre

    # conversion des monnaies en euros
    rates['rates']['EUR'] = 1.0  # ajour de EUR pour pouvoir utiliser map()
    df['money_eur'] = df['money_eur'] * df['currency'].map(rates['rates'])

    # suppression des emails absents
    df = df.dropna(subset=['email'])

    # suppression des blancs
    df['email'] = df['email'].str.strip()

    # emails avec noms de domaine valides
    df = df.loc[df['email'].str.contains('.+@[A-Za-z0-9_\-.]+\.[A-Za-z]{2,}')]
    
    # extraction des aliases (char +)
    tab_email = df['email'].str.extract('([^+]+)(\+.*)?(@.+)').sort_values(0)
    # suppression des aliases (char +)
    df['email'] = tab_email[0] + tab_email[2]

    # suppression des emails en double, on conserve la première ligne
    df = df.drop_duplicates(subset=['email'])

    # assignation des préférences
    tab_preference = df['preference'].str.get_dummies(sep='/')
    df[tab_preference.columns] = tab_preference.astype(bool)

    return df

In [ ]:
# run
import pandas as pd

df = pd.read_csv('people.csv')
print(df.shape)

df = clean_people(df)
print(df.shape)

#### Etudier les multiples options de read_csv

En particulier:

<pre>
pd.read_csv(
    <strong>filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]],</strong>
    <strong>sep=',',</strong>
    delimiter=None,
    <strong>header='infer',</strong>
    <strong>names=None,</strong>
    <strong>index_col=None,</strong>
    <strong>usecols=None,</strong>
    squeeze=False,
    prefix=None,
    mangle_dupe_cols=True,
    <strong>dtype=None,</strong>
    <strong>engine=None,</strong>
    <strong>converters=None,</strong>
    true_values=None,
    false_values=None,
    skipinitialspace=False,
    <strong>skiprows=None,</strong>
    <strong>skipfooter=0,</strong>
    nrows=None,
    <strong>na_values=None,</strong>
    <strong>keep_default_na=True,</strong>
    na_filter=True,
    verbose=False,
    skip_blank_lines=True,
    parse_dates=False,
    infer_datetime_format=False,
    keep_date_col=False,
    date_parser=None,
    dayfirst=False,
    cache_dates=True,
    iterator=False,
    <strong>chunksize=None,</strong>
    compression='infer',
    <strong>thousands=None,</strong>
    <strong>decimal: str = '.',</strong>
    lineterminator=None,
    quotechar='"',
    quoting=0,
    doublequote=True,
    escapechar=None,
    comment=None,
    encoding=None,
    dialect=None,
    error_bad_lines=True,
    warn_bad_lines=True,
    delim_whitespace=False,
    low_memory=True,
    memory_map=False,
    float_precision=None,
)
</pre>

In [ ]:
#pd.read_csv?

#### Analyse automatique avec pandas_profiling

**ATTENTION, il vaut mieux installer `pandas_profiling` dans un nouvel environnement**

<pre>
conda create --name profiling

activate profiling OU conda activate profiling

conda install -c conda-forge pandas-profiling
</pre>

In [ ]:
# profiling raw people
from pandas_profiling import ProfileReport

df = pd.read_csv('people.csv')

profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)

profile.to_file("people.html")

In [ ]:
# profiling clean people
profile = ProfileReport(clean_people(df), title='Pandas Profiling Report', explorative=True)

profile.to_file("clean_people.html")